In [35]:
from py_ball import league

# 1) Re-use the dict you built for requests.get(...)
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.3 Safari/605.1.15"
    ),
    "Referer": "https://stats.nba.com/",
    "Accept": "application/json, text/plain, */*",
}

# 2) Pass that dict — not the Response — into League(...)
league_id = '00'  # WNBA
franchises = league.League(
    headers=headers,
    endpoint="leaguegamelog",
    league_id=league_id,
    season_id="2023-2024",
    season_type="Playoffs"
)

# 3) Now you can inspect your data:
print(franchises.data)

{'LeagueGameLog': [{'SEASON_ID': '42024', 'PLAYER_ID': 1629614, 'PLAYER_NAME': 'Andrew Nembhard', 'TEAM_ID': 1610612754, 'TEAM_ABBREVIATION': 'IND', 'TEAM_NAME': 'Indiana Pacers', 'GAME_ID': '0042400404', 'GAME_DATE': '2025-06-13', 'MATCHUP': 'IND vs. OKC', 'WL': 'L', 'MIN': 36, 'FGM': 4, 'FGA': 9, 'FG_PCT': 0.444, 'FG3M': 2, 'FG3A': 4, 'FG3_PCT': 0.5, 'FTM': 0, 'FTA': 0, 'FT_PCT': None, 'OREB': 0, 'DREB': 2, 'REB': 2, 'AST': 2, 'STL': 2, 'BLK': 1, 'TOV': 4, 'PF': 4, 'PTS': 10, 'PLUS_MINUS': 3, 'FANTASY_PTS': 20.4, 'VIDEO_AVAILABLE': 1}, {'SEASON_ID': '42024', 'PLAYER_ID': 1630167, 'PLAYER_NAME': 'Obi Toppin', 'TEAM_ID': 1610612754, 'TEAM_ABBREVIATION': 'IND', 'TEAM_NAME': 'Indiana Pacers', 'GAME_ID': '0042400404', 'GAME_DATE': '2025-06-13', 'MATCHUP': 'IND vs. OKC', 'WL': 'L', 'MIN': 29, 'FGM': 7, 'FGA': 12, 'FG_PCT': 0.583, 'FG3M': 2, 'FG3A': 5, 'FG3_PCT': 0.4, 'FTM': 1, 'FTA': 4, 'FT_PCT': 0.25, 'OREB': 1, 'DREB': 6, 'REB': 7, 'AST': 1, 'STL': 1, 'BLK': 1, 'TOV': 0, 'PF': 3, 'PTS': 

In [36]:
import pandas as pd

season_df = pd.DataFrame(franchises.data["LeagueGameLog"])

In [37]:
season_df['GAME_DATE'] = pd.to_datetime(season_df['GAME_DATE'])

In [38]:
season_df[season_df.PLAYER_NAME.str.contains("Shai")].sort_values(by='GAME_DATE', ascending=False).head(10).reset_index(drop=True)

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE
0,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400404,2025-06-13,OKC @ IND,W,...,3,0,3,1,2,4,35,0,48.6,1
1,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400403,2025-06-11,OKC @ IND,L,...,8,4,0,3,6,2,24,-3,42.6,1
2,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400402,2025-06-08,OKC vs. IND,W,...,5,8,4,1,2,3,34,5,65.0,1
3,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400401,2025-06-05,OKC vs. IND,L,...,5,3,3,0,2,2,38,3,55.5,1
4,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400315,2025-05-28,OKC vs. MIN,W,...,7,8,2,0,2,2,34,23,58.4,1
5,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400314,2025-05-26,OKC @ MIN,W,...,9,10,1,0,5,4,40,5,63.8,1
6,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400313,2025-05-24,OKC @ MIN,L,...,2,6,0,1,4,2,14,-32,24.4,1
7,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400312,2025-05-22,OKC vs. MIN,W,...,3,8,3,0,1,5,38,25,61.6,1
8,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400311,2025-05-20,OKC vs. MIN,W,...,5,9,3,0,2,1,31,22,57.5,1
9,42024,1628983,Shai Gilgeous-Alexander,1610612760,OKC,Oklahoma City Thunder,0042400227,2025-05-18,OKC vs. DEN,W,...,3,4,3,1,0,2,35,32,56.6,1
